In [1]:
from tifffile import *
import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import label, regionprops, find_contours
import time
import cv2
%matplotlib inline


In [2]:
tiff_path = '/media/jamesrowland/DATA/plab/j031/4/2019-02-28_J031_t-004.tif'
tiff = imread(tiff_path)
tiff.shape

(1942, 512, 512)

In [3]:
#stim occured from frame 60-68
stim_start = 60
stim_end = 68

In [4]:
def threshold(tiff, stim_start, sigma=4):
    
    '''
    tiff: tiff stack 
    stim_start: frame on which the stimulus started
    sigma: sigma value over which to threshold pixels

    thresh: thresholded single frame
    '''
         
    base_vals = tiff[stim_start-10:stim_start-2, :, :]
    base_mean = np.mean(base_vals, 0)
    base_std =  np.std(base_vals, 0)
    thresh = base_mean + base_std*sigma
    
    return thresh 

In [5]:
def region_widths(frame):
    
    '''
    finds regions of connected pixels in a frame and their widths
    returns:
    labelled: array of frame dimensions with each pixels region labelled
    widths: the width (x len) of each region
    '''
    
    labelled, num_labels = label(frame, connectivity=1, return_num=True)
    regions = regionprops(labelled)

    widths = []

    for props in regions:
        coords = props['Coordinates']
        width = max(coords[:,1]) - min(coords[:,1])
        widths.append(width)
        
    widths = np.array(widths)  
    
    return labelled, widths      

In [6]:
def process_frame(frame_bin, frame, width_thresh):
        
    t1 = time.time()
    
    labelled, widths = region_widths(frame_bin)
    
    processed_frame = np.copy(frame)

    for group in np.where(widths<width_thresh)[0]:
        
        if group==0:continue
          
        contam_x,contam_y = np.where(labelled==group)
        
        processed_frame[contam_x, contam_y] = 0
     
    imsave('prco.tiff', processed_frame.astype('int16'))

    return processed_frame

In [7]:
def main(tiff, stim_start, stim_end, width_thresh=10):
    
    thresh = threshold(tiff, stim_start)
    
    for frame_idx in range(stim_start, stim_end):
        
        t1 = time.time()

        frame = tiff[frame_idx, :, :]

        frame_bin = np.greater(frame, thresh).astype('int')
        processed_frame = process_frame(frame_bin, frame, width_thresh)

        tiff[frame_idx, :, :] = processed_frame
        
        
    return tiff

In [8]:
t1 = time.time()
processed_tiff = main(tiff, stim_start, stim_end)
t2 = time.time()
t2-t1

127.23767733573914

In [9]:
imsave('original_stack.tiff', tiff[0:200,:,:])
imsave('processed_stack.tiff', processed_tiff[0:200,:,:])